## Applied Machine Learning: Homework 7, Problem 2
Group members:  
* Eloise Rosen, NetID: eloiser2 
* Mark Berman, NetID: markcb2

In [1]:
import pandas as pd
import numpy as np

## Image segmentation using EM 
You can segment an image using a clustering method - each segment is the cluster center to which a pixel belongs. In this exercise, you will represent an image pixel by its r, g, and b values (so use color images!). Use the EM algorithm applied to the mixture of normal distribution model lectured in class to cluster image pixels, then segment the image by mapping each pixel to the cluster center with the highest value of the posterior probability for that pixel. You must implement the EM algorithm yourself (rather than using a package). Test images are here, and you should display results for all three of them. Till then, use any color image you care to.

Segment each of the test images to 10, 20, and 50 segments. You should display these segmented images as images, where each pixel's color is replaced with the mean color of the closest segment

We will identify one special test image. You should segment this to 20 segments using five different start points, and display the result for each case. Is there much variation in the result?

### Citations
My citations are included as comments in my code. They are repeated below for convenience.  
* 